In [ ]:
# store start time to get execution time of entire script
import time
start_time = time.time()

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import csv

df = pd.read_csv('data_cleaned.csv')
df.head()

In [ ]:
df['Address'] = df['agyaddr'] + ', ' + df['City'] + ', ' + df['State']
addresses = list(df['Address'].unique())
len(addresses)

In [ ]:
%%time

from opencage.geocoder import OpenCageGeocode
from tqdm.notebook import tqdm

# get longitude and latitude for each unique address
key = '2d4ad4cfcdc44021abfe1e347daa6dc7'
geocoder = OpenCageGeocode(key)

geocoded_addrs = []

for addr in tqdm(addresses):
    results = geocoder.geocode(addr)
    lat = results[0]['geometry']['lat']
    lng = results[0]['geometry']['lng']
    geocoded_addrs.append([addr, lat, lng])
    # print(addr + ' -> (' + str(lat) + ', ' + str(lon) + ')')

In [ ]:
geocoded_addrs_df = pd.DataFrame(geocoded_addrs, columns=['Address', 'lat', 'lng'])
geocoded_addrs_df.head()

In [ ]:
geocoded_addrs_df.isna().sum()

In [ ]:
%%time

import urllib, json, requests
import numpy as np

# get block_fips code based on longitude and latitude
def api_call(row):
    lat = row['lat']
    # http://data.fcc.gov/api/block/find?format=json&latitude=28.35975&longitude=-81.421988&showall=true
    lng = row['lng']
    URL = "https://geo.fcc.gov/api/census/block/find?format=json&latitude=" + str(lat) + "&longitude=" + str(lng)
    with urllib.request.urlopen(URL) as url:
        data = json.loads(url.read().decode())
    try:
        return int(data['County']['FIPS']), int(data['Block']['FIPS']), data['State']['name']
    except TypeError: # response failed to get county fips
        return np.nan, np.nan, np.nan

geocoded_addrs_df['state_name'] = ''
for ind,row in tqdm(geocoded_addrs_df.iterrows(), total=geocoded_addrs_df.shape[0]):
    county_FIPS, block_FIPS, state_name = api_call(row)
    geocoded_addrs_df.set_value(ind, 'county_FIPS', county_FIPS)
    geocoded_addrs_df.set_value(ind, 'block_FIPS', block_FIPS)
    geocoded_addrs_df.set_value(ind, 'state_name', state_name)
    
geocoded_addrs_df.head()

In [ ]:
geocoded_addrs_df.isna().sum() # how many nulls per column

In [ ]:
# re-merge patient data and geocoded addresses by the address feature
df = pd.merge(df, geocoded_addrs_df, on=['Address'], how='left')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.to_csv('data_geocoded.csv')

In [ ]:
# print out total notebook execution time
total_seconds = int(time.time() - start_time)
minutes = total_seconds // 60
seconds = total_seconds % 60
print("--- " + str(minutes) + " minutes " + str(seconds) + " seconds ---")